In [ ]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
from datetime import datetime 
import json 


import sys 
sys.path.append('../../../')
from bin.models.trends.utility import TrendUtility, DataUtility
from bin.models.trends.Detect_class import Classifier, ClassificationLog 
from bin.models.trends.result_signals import VolumeOpenInterestWorksheet, WorksheetEntry
from bin.main import get_path 
from main import Manager


import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


from typing import Any, List, Dict, Optional, Union, Tuple
from dataclasses import dataclass 

@dataclass
class TrendResult:
    stock: str
    name: str 
    trend_direction: str
    seasonality: float
    change_point: float 
    valley: Any 
    peaks: Any
    metric_status: str
    blowoff_flag: bool

    def to_dict(self) -> Dict:
        return {
            "stock": self.stock,
            "name": self.name,
            "trend_direction": self.trend_direction,
            "seasonality": self.seasonality,
            "change_point": self.change_point,
            "valley": self.valley,
            "peaks": self.peaks,
            "metric_status": self.metric_status,
            "blowoff_flag": self.blowoff_flag
        }
@dataclass
class ResultsWorksheet:
    worksheets: List[VolumeOpenInterestWorksheet]
    timestamp: datetime
    total_bullish: int = None
    total_bearish: int = None

    def __post_init__(self):
        self.total_bullish = sum(ws.bullish_signals for ws in self.worksheets)
        self.total_bearish = sum(ws.bearish_signals for ws in self.worksheets)

    def to_df(self) -> pd.DataFrame:
        if not self.worksheets:
            return pd.DataFrame()
        return pd.concat([ws.to_df() for ws in self.worksheets], ignore_index=True)

    def to_dict(self) -> Dict:
        return {
            "timestamp": self.timestamp.isoformat(),
            "total_bullish": self.total_bullish,
            "total_bearish": self.total_bearish,
            "worksheets": [ws.to_df().to_dict(orient='records') for ws in self.worksheets]
        }
    

class TResults:
    def __init__(self, connections: Dict|str, lookback_days: int = 90, window_size: int = 30, period: int = 3):
        self.lookback_days = lookback_days
        self.window_size = window_size
        self.period = period
        self.data_utility = DataUtility(connections)
        self.trend_utility = TrendUtility()
        self.stocks = self.data_utility.stocks
        self.all_worksheets = []
        self.returns_df = []
        self.results_worksheet = ResultsWorksheet(worksheets=[], timestamp=datetime.now())

    def get_aligned_data(self, stock: str, backtest_date: Optional[str] = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
        stock_data = self.data_utility.get_aligned_data(stock, backtest_date=backtest_date)
        if backtest_date is not None: 
            self.returns_df.append(self.data_utility.returns_df.copy())
        return stock_data
    
    def analyze_stock_trend(self, stock: str, backtest_date: Optional[str] = None): 
        stock_data = self.get_aligned_data(stock, backtest_date=backtest_date)
        trend_results = self.trend_utility.analyze_trends(stock_data)
        return trend_results
    
    def trend_result_to_df(self, trend_result: TrendResult) -> pd.DataFrame:
        d = trend_result.copy()
        date = list(d.keys())[0]
        short_term = d[date]['short_term']
        long_term = d[date]['long_term']
        ytd = d[date]['ytd']
        st = pd.DataFrame(short_term).T
        lt = pd.DataFrame(long_term).T
        yt = pd.DataFrame(ytd).T
        st['timeframe'] = 'short_term'
        lt['timeframe'] = 'long_term'
        yt['timeframe'] = 'ytd'
        out = pd.concat([st, lt, yt]).sort_index().reset_index().rename(columns = {'index': 'name'})
        out.insert(0, 'date', date)
        out['stock'] = trend_result.stock
        return out
    
    def worksheet_entry(self, stock: str, trend_results: Dict) -> VolumeOpenInterestWorksheet:
        df = self.trend_result_to_df(trend_results)
        date = df.date.iloc[0]
        price = df[df.name == 'price']['value'].iloc[0]
        returns = df[df.name == 'returns']['value'].iloc[0]
        pass

In [6]:
connections = get_path('../../../')
tr = TResults(connections, lookback_days=90, window_size=30, period=3)

2025-06-09 14:38:38,237 - bin.options.optgd.db_connect - INFO - Options DB Initialized
2025-06-09 14:38:38,238 - bin.options.optgd.db_connect - WARNING - You currently have 8 connections open.
2025-06-09 14:38:38,241 - INFO - PriceDB Initialized successfully at 2025-06-09 14:38:38.241961
2025-06-09 14:38:38,241 - bin.price.db_connect - INFO - PriceDB Initialized successfully at 2025-06-09 14:38:38.241961
2025-06-09 14:38:38,242 - INFO - Established 3 database connections
2025-06-09 14:38:38,242 - bin.price.db_connect - INFO - Established 3 database connections
2025-06-09 14:38:38,295 - bin.alerts.options_alerts - INFO - Notifications instance initialized.
2025-06-09 14:38:38,296 - bin.options.optgd.db_connect - INFO - Options DB Initialized
2025-06-09 14:38:38,296 - bin.options.optgd.db_connect - WARNING - You currently have 8 connections open.


In [7]:
sd = tr.get_aligned_data('aapl')

In [8]:
d = tr.analyze_stock_trend('aapl')

In [12]:

date = list(d.keys())[0]
short_term = d[date]['short_term']
long_term = d[date]['long_term']
ytd = d[date]['ytd']
st = pd.DataFrame(short_term).T
lt = pd.DataFrame(long_term).T
yt = pd.DataFrame(ytd).T
st['timeframe'] = 'short_term'
lt['timeframe'] = 'long_term'
yt['timeframe'] = 'ytd'
out = pd.concat([st, lt, yt]).reset_index().rename(columns = {'index': 'name'})
out.insert(0, 'date', date)

In [13]:
out.name.unique()

array(['price', 'returns', 'stock_volume', 'options_volume', 'oi',
       'pcr_volume', 'pcr_oi', 'total_vol_oi', 'atm_iv', 'call_volume',
       'put_volume'], dtype=object)

In [14]:
date = out.date.iloc[0]
price = out[out.name == 'price']['value'].iloc[0]
returns = out[out.name == 'returns']['value'].iloc[0]


In [15]:
out

,date,name,value,trend,seasonality,slope,status,blowoff,peaks,valleys,change_point,timeframe
0,2025-06-06,price,204.910004,up,high,-0.001159,Average,False,203.270004,200.630005,200.630005,short_term
1,2025-06-06,returns,0.021333,down,high,-0.000543,Blowoff,True,0.007784,-0.010798,-0.010798,short_term
2,2025-06-06,stock_volume,26376516,up,high,-0.004245,Low,False,55126100,35423300,55126100,short_term
3,2025-06-06,options_volume,784200.0,down,high,-0.00238,Average,False,848762.0,423876.0,423876.0,short_term
4,2025-06-06,oi,5457085.0,down,high,0.003171,Blowoff,True,5253271.0,4930810.0,4930810.0,short_term
5,2025-06-06,pcr_volume,0.74626,down,normal,-0.007385,Average,False,0.745707,0.598386,0.598386,short_term
6,2025-06-06,pcr_oi,0.73751,up,normal,-0.003781,Low,False,0.756942,0.741192,0.756942,short_term
7,2025-06-06,total_vol_oi,6241285.0,down,high,0.002488,Blowoff,True,5777180.0,5354686.0,5354686.0,short_term
8,2025-06-06,atm_iv,0.27558,up,low,-0.009809,Low,False,0.298239,0.28335,0.28335,short_term
9,2025-06-06,call_volume,449074.0,down,high,-0.001029,Average,False,320013.0,264569.0,264569.0,short_term
